# Install and Import Dependencies

In [18]:
!pip uninstall opencv-python-headless --yes

Found existing installation: opencv-python-headless 4.5.1.48
Uninstalling opencv-python-headless-4.5.1.48:
  Successfully uninstalled opencv-python-headless-4.5.1.48


In [47]:
!pip install mediapipe opencv-python==4.5.1.48 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6087 sha256=1614f4c16723c97bd782321346f2221a418e96b37d253e43c04857b781caea64
  Stored in directory: C:\Users\beard\AppData\Local\Temp\pip-ephem-wheel-cache-barmil8y\wheels\30\33\46\5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# Make Detections

# Determining Joints

# Calculate Angles

# Curl Counter